In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import linregress

# Fáza 2 - Predspracovanie údajov

## 2.1 Realizácia predspracovania dát

### A/B - Rozdelenia dát na trénovaciu a testovaciu množinu

### dáta načítaváme zo súboru, ktorý sme vytvorili v EDA časti, na základe našich pozorovaní (asi bude treba potrebné pridať samostatné funkcie na to ...)

In [9]:
# nacitanie dat
df = pd.read_csv('merged_data.csv')
print(f"shape of df: {df.shape}")
# df.head()

shape of df: (15414, 10)


### Rozdelíme dáta v pomere 80:20 (trénovacia/testovacia množina). Tento pomer vychádza z predpokladu, že pri dostatočne veľkom množstve dát, máme pri rozdelení dostatok dát na trénovanie a efektívne učenie modelu, zatiaľ čo testovacia množina bude dostatočne veľká pre overenie modelu. Tento pomer je bežný a vhodný pre väčšinu úloh strojového učenia.

In [ ]:
# funkcia na rozdelenie dat
TRAIN = 0.8

def split_data(df):
    row, col = df.shape
    train_size = int(row * TRAIN)
    train_df = df[:train_size]
    test_df = df[train_size:]
    return train_df, test_df

# NOTE: mozno upravit na random vyber dat (da sa s mnimi pracovat jak stale , len chapeme sa xd , zeby to bolo viac random)
# + ze akoze mozno ten datum by som pridal (ja pridam), lebo piodla veku sa tiez oze ovplivnovat mwra

In [ ]:
df_train, df_test = split_data(df)

### ďalej pracujeme s trénovacím datasetom